In [ ]:
import bilby
import numpy as np
from scipy import signal
import math
import seaborn as sns

import h5py 
from pathlib import Path

import matplotlib.pyplot as plt

def mod_sinusoid(time, A_0, A_am, A_fm, omega_0, omega_ax, phi_c, phi_ax):
    
    amplitude = A_0 + A_am * np.cos(1 * omega_ax * time - phi_ax)
    
    omega_c = omega_0 + A_fm * np.sin(1 * omega_ax * time - phi_ax - math.pi)
    
    phase = omega_c * time - phi_c
    
    return amplitude * np.exp(1j * phase) * np.exp(-1j * omega_lo * time)

def select_index(meta_dict, theta, rad, energy):
    
    #unique = [np.sort(np.unique(meta_dict[key])) for i, key in enumerate(meta_dict)]
    #grids = np.meshgrid(*unique)
    
    keys = list(meta_dict.keys())
    
    mask = abs(meta_dict[keys[0]] - theta) < 1e-6
    mask = np.logical_and(mask, abs(meta_dict[keys[1]] - rad) < 1e-6)
    mask = np.logical_and(mask, abs(meta_dict[keys[2]] - energy) < 1e-6)
    
    return np.argwhere(mask).squeeze()


datapath = Path.home()/'group'/'project'/'datasets'/'data'/'220414_dl_grid_84to90_1to45mm.h5'

h5file = h5py.File(datapath, 'r')

meta_keys = ['theta_min', 'x_min', 'energy']

meta_dict = {}
for i, key in enumerate(meta_keys):
    meta_dict[key] = h5file['meta'][key][:]
    
    
    
energy = 18550
pitch = np.linspace(87.0, 88.0, 21)
radius = 0.01

time_series_list = []

for i, angle in enumerate(pitch):
    
    time_series_list.append(h5file['data'][select_index(meta_dict, angle, radius, energy), :])

time_series_list = np.array(time_series_list)
frequency_series_list = np.fft.fftshift(np.fft.fft(time_series_list, axis=-1, norm='forward'), axes=-1)
peak threshold = 1e-10

peaks = {}

for i in range(frequency_series_list.shape[0]):
    
    print(signal.find_peaks(frequency_series_list[i, :], ))

sns.set_theme(context='talk', style='whitegrid',)
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(abs(frequency_series_list[0]))
ax.set_ylim(0.0, 0.4e-9)


